<h4>Defining an Action</h4>

This is building off of ([here](https://docs.ros.org/en/galactic/Tutorials/Intermediate/Creating-an-Action.html)) and defining it ([here](https://docs.ros.org/en/galactic/Tutorials/Intermediate/Writing-an-Action-Server-Client/Py.html))  If you want the an exact replication, go [here](ActionTutorial.ipynb).  The first step is to define it by creating the requisite paths, then folders and then create the package defining the action.

In [ ]:
import os
import subprocess

home = os.path.expanduser('~')
workspace = home +'/ros2_ws/'
sourceFiles = workspace + 'src/'
actionName = 'timer'

os.chdir(home)

if not os.path.exists(sourceFiles):
    os.makedirs(sourceFiles)
os.chdir(sourceFiles)
if not os.path.exists(sourceFiles + actionName):
    reply = subprocess.run(['ros2','pkg','create',actionName],capture_output=True) #same as !ros2 pkg create action_tutorials_interfaces
    os.chdir(sourceFiles + actionName)
    os.mkdir('action')

Next we set up the action definition file.  This defines the data types for *Resquest*, *Result*, and *Feedback*: the three parts to every action.  In this case, request (only one parameter: order) is a 32 bit integer, the result (only one parameter: sequence) is an array of 32 bit integers, and the feedback you get when quering it while it is running (partial_sequence) is a similar array.  So move to the action folder and then write the definition file.

In [ ]:
os.chdir(sourceFiles + actionName + '/action')

In [ ]:
%%writefile Timer.action

int32 duration
---
int32 done
---
int32 sec_left

We then have to edit the CMakeList - adding the package *rosidl_generate_interafaces* and then generating the ros2 interface for the action we just defined above.
```bash
find_package(rosidl_default_generators REQUIRED)

rosidl_generate_interfaces(${PROJECT_NAME}
  "action/Timer.action"
)

```

In [ ]:
os.chdir(sourceFiles + actionName)

In [ ]:
%%writefile CMakeLists.txt
cmake_minimum_required(VERSION 3.8)
project(timer)

if(CMAKE_COMPILER_IS_GNUCXX OR CMAKE_CXX_COMPILER_ID MATCHES "Clang")
  add_compile_options(-Wall -Wextra -Wpedantic)
endif()

# find dependencies
find_package(ament_cmake REQUIRED)
# uncomment the following section in order to fill in
# further dependencies manually.
# find_package(<dependency> REQUIRED)

if(BUILD_TESTING)
  find_package(ament_lint_auto REQUIRED)
  # the following line skips the linter which checks for copyrights
  # uncomment the line when a copyright and license is not present in all source files
  #set(ament_cmake_copyright_FOUND TRUE)
  # the following line skips cpplint (only works in a git repo)
  # uncomment the line when this package is not in a git repo
  #set(ament_cmake_cpplint_FOUND TRUE)
  ament_lint_auto_find_test_dependencies()
endif()

find_package(rosidl_default_generators REQUIRED)

rosidl_generate_interfaces(${PROJECT_NAME}
  "action/Timer.action"
)

ament_package()


We then edit the package xml so that it has the new dependencies for the action
```bash
<buildtool_depend>rosidl_default_generators</buildtool_depend>

<depend>action_msgs</depend>

<member_of_group>rosidl_interface_packages</member_of_group>
```

In [ ]:
os.chdir(sourceFiles + actionName)

In [ ]:
%%writefile package.xml
<?xml version="1.0"?>
<?xml-model href="http://download.ros.org/schema/package_format3.xsd" schematypens="http://www.w3.org/2001/XMLSchema"?>
<package format="3">
  <name>timer</name>
  <version>0.0.0</version>
  <description>TODO: Package description</description>
  <maintainer email="parallels@todo.todo">parallels</maintainer>
  <license>TODO: License declaration</license>

  <buildtool_depend>ament_cmake</buildtool_depend>
    
  <buildtool_depend>rosidl_default_generators</buildtool_depend>
  <depend>action_msgs</depend>
  <member_of_group>rosidl_interface_packages</member_of_group>
    
  <test_depend>ament_lint_auto</test_depend>
  <test_depend>ament_lint_common</test_depend>

  <export>
    <build_type>ament_cmake</build_type>
  </export>
</package>


Lastly we have to build it by running *colcon build* in the workspace directory

In [ ]:
import subprocess
os.chdir(home + '/ros2_ws')
reply = subprocess.run(['colcon','build'],capture_output=True)

In [ ]:
reply

Now you have to go to the terminal and run 
```bash
cd ~/ros2_ws
export ROS_DOMAIN_ID=0
. install/setup.bash
ros2 interface show timer/action/Timer
```
This will test the action definition

Now that you have defined the action - you need to write some code - I recommend starting with the server - [here](103f-ROS_action_server.ipynb)